In [ ]:
%load_ext autoreload
%autoreload 2

# Exploring Fractal Sets

With ths notebook, you can navigate through a fractal set and render pictures from the plot. First some imports and definitions: You also can define a complex polynomial here, that will applied to the numbers in the complex plane.

In [ ]:
import numpy as np
import slfractals as slf
from matplotlib import pyplot as plt
from matplotlib import cm

def poly(z, c):
    return z**2 + c

* Define the polynomial in terms of variable (``z``) and initial value (``c``) that is going to be iterated on ``z``.
* You cannot render in parallel with ``poly``, you must use a pre-defined function from ``slf.polynomials``. Rendering in serial way is fine.

We can now run the iteration and investigate the occurring values for the iteration in the selected portion of the complex plain. First, we set some parameters for the preview.
* List of colormaps: https://matplotlib.org/tutorials/colors/colormaps.html

In [ ]:
params = {
    "max_iter": 200,
    "max_value": 2,
    "xb": (-1.8, 0.67),
    "yb": (-1.2, 1.2),
    "reswidth": 301,
    "cmap": cm.RdYlGn.reversed()
}

## Preview

Navigate the figure (zoom and pan) to explore the fractal set. A recomputation will be issued on the fly. Experiment with the parameters ``max_iter`` and ``max_value`` to get a feeling for what they do.

In [ ]:
%matplotlib notebook

def onzoom(event, ax):
    xb = ax.get_xlim()
    yb = ax.get_ylim()
    C = slf.get_grid(xb, yb, resw=params["reswidth"])
    G = slf.serial_compute(poly, C, max_value=params["max_value"], max_iter=params["max_iter"], colorexp=2)
    ax.clear()
    ax.imshow(
        G,
        cmap=params["cmap"],
        extent=[*xb, *yb],
        aspect="equal"
    )

fig = plt.figure(num=1, figsize=(4.5, 3.5), dpi=200)
fig.clear()
ax = fig.add_subplot(111)
ax.set_xlim(*params["xb"])
ax.set_ylim(*params["yb"])
onzoom(_, ax)
cid = fig.canvas.mpl_connect("button_release_event", lambda event: onzoom(
    event,
    ax
))

## Rendering

You can set different parameters for rendering here:

In [ ]:
renderwidth = 1920
print_max_iter = 300
xb = ax.get_xlim()
yb = ax.get_ylim()
print(xb, yb)

### Serial rendering

Use rendering on one core, if you have defined a custom polynomial in ``poly``.

In [ ]:
%%timeit -r 1 -n 1
G = slf.serial_compute(
    poly,
    slf.get_grid(xb, yb, resw=renderwidth, ratio="16:9"),
    max_value=params["max_value"],
    max_iter=print_max_iter
)
slf.render_picture(G, "files", cmap=params["cmap"])

### Parallel rendering 

Parallel rendering on multiple cores is restricted to pre-defined polynomials from ``slf.polynomials``. It is an issue with ``multiprocessing.Pool`` not allowing being able to pickle lambda functions or functions defined in the notebook. Adapt ``nproc`` to the available computing cores.

In [ ]:
%%timeit -r 1 -n 1
G = slf.parallel_compute(
    slf.mandel,
    slf.get_grid(xb, yb, resw=renderwidth, ratio="16:9"),
    max_value=params["max_value"],
    max_iter=print_max_iter,
    nproc=2
)
slf.render_picture(G, "files", cmap=params["cmap"])

In [ ]:
from matplotlib.cm import inferno
inferno(100)